In [5]:
import numpy as np
import random

num_entities = 10
num_validators = 500000
num_slots = 300000
num_builders = 10
fixed_reward = 0.025
sigma_mev_reward = 1
split_threshold = 32
validator_share = 0.95
builder_share = 0.05

# Define the mean MEV rewards for each builder
builder_mev_mean_rewards = [0.5] + [0.3]*2 + [0.1]*(num_builders-3)

# Compute the mean and standard deviation of the underlying normal distribution
builder_normal_mean_rewards = [np.log(mean_reward) for mean_reward in builder_mev_mean_rewards]

# Create validator_entity_mapping and initialize rewards
validator_entity_mapping = [i % num_entities for i in range(num_validators)]
validator_rewards = np.zeros(num_validators)
entity_rewards = np.zeros(num_entities)
builder_rewards = np.zeros(num_builders)

# Create lists of validators per entity
validators_per_entity = [[] for _ in range(num_entities)]
for validator_idx, entity in enumerate(validator_entity_mapping):
    validators_per_entity[entity].append(validator_idx)

# Run the simulation
for _ in range(num_slots):
    # Select a random validator
    chosen_validator = random.randrange(num_validators)

    # Compute the MEV rewards for each builder
    mev_rewards = np.array([np.random.lognormal(mean=mean_reward, sigma=sigma_mev_reward) for mean_reward in builder_normal_mean_rewards])

    # Validator chooses the highest MEV reward
    max_mev_reward = np.max(mev_rewards)
    chosen_builder = np.argmax(mev_rewards)

    # Assign rewards
    block_reward = fixed_reward
    total_reward = block_reward + max_mev_reward

    validator_rewards[chosen_validator] += validator_share * total_reward
    entity_rewards[validator_entity_mapping[chosen_validator]] += validator_share * total_reward
    builder_rewards[chosen_builder] += builder_share * total_reward

    # Check if we need to spawn a new validator for any entity
    for entity_idx in range(num_entities):
        if entity_rewards[entity_idx] >= split_threshold:
            # Spawn a new validator for this entity
            num_validators += 1
            validator_entity_mapping.append(entity_idx)
            validator_rewards = np.append(validator_rewards, 0)
            validators_per_entity[entity_idx].append(num_validators - 1)

            # Deduct the rewards used to spawn the new validator from the existing validators
            to_deduct = split_threshold
            for validator_idx in validators_per_entity[entity_idx]:
                if to_deduct > 0:
                    deduct_amount = min(validator_rewards[validator_idx], to_deduct)
                    validator_rewards[validator_idx] -= deduct_amount
                    entity_rewards[entity_idx] -= deduct_amount
                    to_deduct -= deduct_amount
                else:
                    break

# Calculate the total value each entity controls
validators_count_per_entity = [len(validators) for validators in validators_per_entity]
entity_values = np.array(validators_count_per_entity) * 32 + entity_rewards

# Print the ranked list of builder profits
sorted_builder_indices = np.argsort(builder_rewards)[::-1]
print("Builder profits:")
for idx in sorted_builder_indices:
    print(f"Builder {idx}: {builder_rewards[idx]}")

# Print the ranked list of entity profits
sorted_entity_indices = np.argsort(entity_values)[::-1]
print("Entity profits:")
for idx in sorted_entity_indices:
    print(f"Entity {idx}: {entity_values[idx]}")


Builder profits:
Builder 0: 8981.070966007268
Builder 2: 3752.4189974708047
Builder 1: 3739.0380542369912
Builder 5: 392.8422294901583
Builder 6: 383.2816037705919
Builder 3: 380.6362346591661
Builder 7: 379.5323165480531
Builder 4: 379.35448830932677
Builder 9: 374.8115839462319
Builder 8: 371.34684517581786
Entity profits:
Entity 0: 1636810.7186194826
Entity 8: 1636620.4148789519
Entity 3: 1636596.7211214262
Entity 9: 1636592.9235369773
Entity 1: 1636493.129576316
Entity 2: 1636426.4658983606
Entity 7: 1636377.647147057
Entity 5: 1636067.4873048214
Entity 6: 1635940.9381750282
Entity 4: 1635625.886814251
